## Purpose

To compute the ML performance of the proposed method in near real-time.

In [1]:
import os 
import sys
import numpy as np
from collections import defaultdict
import CAN_objects.aid_message
import matplotlib.pyplot as plt


actt_path = os.path.join(os.path.join(os.path.expanduser("~"), "Projects", "CAN", "actt"))
os.chdir(actt_path)
sys.path.insert(0, "src") # add src folder to path so that files from this folder can be imported

from generalFunctions import unpickle
import subprocess

import importlib
importlib.reload(CAN_objects.aid_message)
from init_cancapture_from_canlog import init_cancap
import json
import seaborn as sns
import pandas as pd

from CAN_objects.capture import MappedCapture, MatchedCapture
import math
from scipy.cluster.hierarchy import single, complete, average, ward, dendrogram, linkage, fcluster

from pprint import pprint
from sklearn.metrics.cluster import normalized_mutual_info_score

from clusim.clustering import Clustering, remap2match
import clusim.sim as sim

import glob
from tqdm import tqdm
import itertools
from scipy.stats import shapiro, mannwhitneyu, ttest_ind, spearmanr
from sklearn.preprocessing import normalize, scale, MinMaxScaler, StandardScaler

from statistics import mode
from bisect import bisect_left

## Enable the Use of Functions From the Detect Repo

In [2]:
# sys.path.insert(0, "/home/cades/Projects/CAN/detect/") # add detect folder to path so that files from this folder can be imported
sys.path.insert(0, "/home/cloud/Projects/CAN/detect/") # add detect folder to path so that files from this folder can be imported
import signal_based_preprocess_functions
print(os.getcwd())

/home/cloud/Projects/CAN/actt


## Functions

In [3]:
def from_capture_to_time_series(cap, ground_truth_dbc_path):
    
    signal_multivar_ts, timepts, aid_signal_tups = signal_based_preprocess_functions.capture_to_mv_signal_timeseries(cap, ground_truth_dbc_path)

    return signal_multivar_ts, timepts, aid_signal_tups


def from_captures_to_time_series(cap_1, cap_2, ground_truth_dbc_path):
        
    signal_multivar_ts_1, timepts_1, aid_signal_tups_1 = signal_based_preprocess_functions.capture_to_mv_signal_timeseries(cap_1, ground_truth_dbc_path)
    signal_multivar_ts_2, timepts_2, aid_signal_tups_2 = signal_based_preprocess_functions.capture_to_mv_signal_timeseries(cap_2, ground_truth_dbc_path)

    return signal_multivar_ts_1, timepts_1, aid_signal_tups_1, signal_multivar_ts_2, timepts_2, aid_signal_tups_2


def remove_constant_signals(signal_multivar_ts):
    return signal_multivar_ts[:, ~np.all(signal_multivar_ts[1:] == signal_multivar_ts[:-1], axis=0)]


def partition_time_series(signal_multivar_ts, window_length, offset):
    
    n = signal_multivar_ts.shape[0]
    i = 0
    partition = []
    
    while (i + window_length) < n:
        partition.append(signal_multivar_ts[i: i + window_length,:])
        i = i + offset
        
    if i != n:
        partition.append(signal_multivar_ts[i:n,:])
        
    return partition
    
    
def process_multivariate_signals(signal_multivar_ts, aid_signal_tups, window_length, offset):
    
    # First dataframe
    # Convert matrix of time series into a dataframe
    df = pd.DataFrame({f"{tup[0]}_{tup[1]}": signal_multivar_ts[:,index] for index, tup in enumerate(aid_signal_tups)})
    # display(df)

    # Remove columns with constant values
    df = df.loc[:, (df != df.iloc[0]).any()] 
    # display(df)
    
    # Stadarization
    # df_standardized = (df-df.mean())/df.std()
    df_standardized = df
    # display(df_standardized)
    
    # Partition of data frames
    n = df_standardized.shape[0]
    i = 0
    partition = []
    
    while (i + window_length) < n:
        partition.append(df_standardized.iloc[i:i + window_length, :])
        i = i + offset
        
    if i != n:
        partition.append(df_standardized.iloc[i:n, :])
        
    return partition


def process_multiple_multivariate_signals(signal_multivar_ts_1, aid_signal_tups_1, signal_multivar_ts_2, aid_signal_tups_2, window_length, offset):
    
    # First dataframe
    # Convert matrix of time series into a dataframe
    df_1 = pd.DataFrame({f"{tup[0]}_{tup[1]}": signal_multivar_ts_1[:,index] for index, tup in enumerate(aid_signal_tups_1)})
    # display(df)
    print(df_1.shape)

    # Remove columns with constant values
    df_1 = df_1.loc[:, (df_1 != df_1.iloc[0]).any()] 
    # display(df)
    
    # Stadarization
    df_1_standardized = (df_1-df_1.mean())/df_1.std()
    # display(df_2_standardized)
    
    # Partition of data frames
    n = df_1_standardized.shape[0]
    i = 0
    partition_1 = []
    
    while (i + window_length) < n:
        partition_1.append(df_1_standardized.iloc[i:i + window_length, :])
        i = i + offset
        
    if i != n:
        partition_1.append(df_1_standardized.iloc[i:n, :])
        
        
    # Second dataframe
    # Convert matrix of time series into a dataframe
    df_2 = pd.DataFrame({f"{tup[0]}_{tup[1]}": signal_multivar_ts_2[:,index] for index, tup in enumerate(aid_signal_tups_2)})
    # display(df)
    print(df_2.shape)

    # Remove columns with constant values
    df_2 = df_2.loc[:, (df_2 != df_2.iloc[0]).any()] 
    # display(df)
    
    # Stadarization
    df_2_standardized = (df_2-df_2.mean())/df_2.std()
    # display(df_2_standardized)
    
    # Partition of data frames
    n = df_2_standardized.shape[0]
    i = 0
    partition_2 = []
    
    while (i + window_length) < n:
        partition_2.append(df_2_standardized.iloc[i:i + window_length, :])
        i = i + offset
        
    if i != n:
        partition_2.append(df_2_standardized.iloc[i:n, :])
        
    return partition_1, partition_2


def upper(df):
    '''Returns the upper triangle of a correlation matrix.
    You can use scipy.spatial.distance.squareform to recreate matrix from upper triangle.
    Args:
      df: pandas or numpy correlation matrix
    Returns:
      list of values from upper triangle
    '''
    try:
        assert(type(df) == np.ndarray)
    except:
        if type(df) == pd.DataFrame:
            df = df.values
        else:
            raise TypeError('Must be np.ndarray or pd.DataFrame')
    mask = np.triu_indices(df.shape[0], k=1)
    
    return df[mask]



def randomized_test_permutations(m1, m2):
    """Nonparametric permutation testing Monte Carlo"""
    np.random.seed(0)
    rhos = []
    n_iter = 100
    true_rho, _ = spearmanr(upper(m1), upper(m2))
    # matrix permutation, shuffle the groups
    m_ids = list(m1.columns)
    m2_v = upper(m2)
    for iter in range(n_iter):
        np.random.shuffle(m_ids) # shuffle list 
        r, _ = spearmanr(upper(m1.loc[m_ids, m_ids]), m2_v)  
        rhos.append(r)
    perm_p = ((np.sum(np.abs(true_rho) <= np.abs(rhos)))+1)/(n_iter+1) # two-tailed test

    return perm_p


def compute_correlation_matrices(partition):
    
    corr_matrices = []

    for df in partition:

        # Remove columns with constant values
        df = df.loc[:, (df != df.iloc[0]).any()] 

        # Compute correlation matrix
        corr_matrices.append(df.corr(method="pearson"))
        
    return corr_matrices


def compute_similarity_from_correlation_matrices(corr_matrices):
    
    similarities = []
    
    for i in range(len(corr_matrices)-1):

        # print("raw: ", corr_matrices[i].shape, corr_matrices[i+1].shape)

        signal_names_1 = corr_matrices[i].columns.values
        signal_names_2 = corr_matrices[i+1].columns.values
        signal_names_intersection = list(set(signal_names_1).intersection(set(signal_names_2)))

        df_1 = corr_matrices[i].loc[signal_names_intersection, signal_names_intersection] 
        df_2 = corr_matrices[i+1].loc[signal_names_intersection, signal_names_intersection]
  
        # print("pro: ", df_1.shape, df_2.shape, "\n")

        similarities.append((df_1.shape[0], spearmanr(upper(df_1), upper(df_2))[0], spearmanr(upper(df_1), upper(df_2))[1]))
        
    return similarities


def compute_similarity_from_multiple_correlation_matrices(corr_matrices_1, corr_matrices_2):
    
    similarities = []
    
    if len(corr_matrices_1) <= len(corr_matrices_2):
        corr_matrices_reference = corr_matrices_1
    else:
        corr_matrices_reference = corr_matrices_2
        
    print(len(corr_matrices_reference))
            
    for i in range(len(corr_matrices_reference)):

        # print("raw: ", corr_matrices[i].shape, corr_matrices[i+1].shape)

        signal_names_1 = corr_matrices_1[i].columns.values
        signal_names_2 = corr_matrices_2[i].columns.values
        signal_names_intersection = list(set(signal_names_1).intersection(set(signal_names_2)))

        df_1 = corr_matrices_1[i].loc[signal_names_intersection, signal_names_intersection] 
        df_2 = corr_matrices_2[i].loc[signal_names_intersection, signal_names_intersection]
  
        # print("pro: ", df_1.shape, df_2.shape, "\n")

        # similarities.append((df_1.shape[0], spearmanr(upper(df_1), upper(df_2))[0], spearmanr(upper(df_1), upper(df_2))[1]))
        
        correlation = spearmanr(upper(df_1), upper(df_2))[0]
        p_value = spearmanr(upper(df_1), upper(df_2))[1]
        
        if p_value > 0.05:
            similarities.append((i, correlation, p_value))
        else:
            similarities.append(i)
            
        
    return similarities


def create_time_intervals(start_time, end_time, window, offset):

    intervals = []
    # window = 100*window
    # offset = 0.1*offset
    
    for i in np.arange(start_time, end_time - window + 1, offset, dtype=float):
        intervals.append((i, i + window))

    if i + window < end_time:
        intervals.append((i + offset, end_time))

    return intervals  


def _bisect_left_mod(signal_times, t_interp):
    idx = bisect_left(signal_times,t_interp)
    if idx > 0:
        return idx - 1
    else:
        return 0


def interpolate_time_series(id_dic, timepts):
    interp_vals = []

    for id, signal_dic in id_dic.items():

        for signal_id, payload in signal_dic.items():

            interp_signals = []

            for t in timepts:

                interp_signals.append(payload[1][_bisect_left_mod(payload[0], t)])
                
            interp_vals.append(interp_signals)

    return np.swapaxes(np.vstack(interp_vals), 0, 1)


def dic_to_mv_signal_timeseries(id_dic, max_start_time, min_end_time, min_hz_msgs=10):
    step = 1000/min_hz_msgs # because SynCAN timestamps are in ms

    timepts = np.arange(max_start_time, min_end_time, step) # to endure we have coverage for the time intervals in each of the IDs

    signal_multivar_ts = interpolate_time_series(id_dic, timepts)

    return signal_multivar_ts, timepts


def reformat_synCAN_dataset(file_name, df, read_df=False):
    if read_df == True:
        df_original = pd.read_csv("/home/cloud/Projects/CAN/actt/data/SynCAN-Dataset/" + file_name)
    else:
        df_original = df

    diff_dic = {}
    id_dic = {}
    start_times = []
    end_times = []
    aid_signal_tups = []

    # unique_ids = df_original.sort_values(by=["ID"], ascending=True)["ID"].unique()
    unique_ids = ["id" + str(i) for i in np.arange(1, 11)]
    # print(unique_ids)

    for count_id, id in enumerate(unique_ids, start=1):

        # Dropping missing columns
        df = df_original[df_original["ID"] == id].dropna(axis=1, how="all")
        # display(df)

        # Time range
        times = df[df["ID"] == id].sort_values(by=["Time"], ascending=True)["Time"].to_numpy()
        start, end = times[0], times[-1]
        start_times.append(start)
        end_times.append(end)
        
        # Time intervals
        diffs = np.diff(times)
        modes = mode(diffs)
        
        diff_dic[id] = diffs
        unique = len(np.unique(diffs))

        # print(f"{id}, unique inter-arrival time: {unique}, mode: {modes}") 
        # print(f"start: {start}, end: {end}, duration (min): {(end-start)/60000}\n")
        interval = np.arange(start, end, 100)
        # print(len(interval), interval)

        # Extract signal names
        df = df.loc[:, df.columns.str.startswith("Signal")]
        signal_names = df.columns.to_numpy()
        # print(signal_names)
        
        # Create dictionary of signals
        signal_dic = {}
        for count_signal, signal_name in enumerate(signal_names, start=0):
            values = df[signal_name].to_numpy()
            signal_dic[signal_name] = [times, values]

            #print(count_id, count_signal)
            aid_signal_tups.append((count_id, count_signal))

        id_dic[id] = signal_dic

    max_start_time = np.ceil(max(start_times))
    min_end_time = np.floor(min(end_times))
        
    return id_dic, aid_signal_tups, max_start_time, min_end_time 


def binary_search(number, intervals):
    # intervals.sort()
    mid = len(intervals)//2  # a//b is in python3. Use len(intervals)/2 for python2
    low, high = intervals[mid]

    # print(number, mid, (low, high), "intervals:", intervals)
    
    if number > high and len(intervals) > 2:
        # print("high", intervals[mid+1:])
        return binary_search(number, intervals[mid+1:])
    elif number < low and len(intervals) > 2:
        # print("low", intervals[:mid])
        return binary_search(number, intervals[:mid])
    elif low <= number <= high:
        return True
    else:
        return False

In [4]:
binary_search(22, [(1,11), (12,15), (17,19), (20,25)])

True

## Process SynCAN

In [7]:
df_original = pd.read_csv("/home/cloud/Projects/CAN/actt/data/SynCAN-Dataset/test_normal.csv")
display(df_original)

df_original =  df_original.sort_values(by=["Time"], ascending=True)
display(df_original)

,Label,Time,ID,Signal1_of_ID,Signal2_of_ID,Signal3_of_ID,Signal4_of_ID
0,0,8.550892e+07,id7,0.055714,0.750000,NaN,NaN
1,0,8.550892e+07,id5,0.327077,0.931964,NaN,NaN
2,0,8.550893e+07,id1,0.621423,0.750000,NaN,NaN
3,0,8.550893e+07,id6,0.921053,0.555556,NaN,NaN
4,0,8.550893e+07,id8,0.215771,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2150047,0,9.000939e+07,id6,0.394737,0.000000,NaN,NaN
2150048,0,9.000939e+07,id3,0.400000,0.806563,NaN,NaN
2150049,0,9.000939e+07,id8,0.156707,NaN,NaN,NaN
2150050,0,9.000939e+07,id1,0.549564,1.000000,NaN,NaN


,Label,Time,ID,Signal1_of_ID,Signal2_of_ID,Signal3_of_ID,Signal4_of_ID
0,0,8.550892e+07,id7,0.055714,0.750000,NaN,NaN
1,0,8.550892e+07,id5,0.327077,0.931964,NaN,NaN
2,0,8.550893e+07,id1,0.621423,0.750000,NaN,NaN
3,0,8.550893e+07,id6,0.921053,0.555556,NaN,NaN
4,0,8.550893e+07,id8,0.215771,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2150047,0,9.000939e+07,id6,0.394737,0.000000,NaN,NaN
2150048,0,9.000939e+07,id3,0.400000,0.806563,NaN,NaN
2150049,0,9.000939e+07,id8,0.156707,NaN,NaN,NaN
2150050,0,9.000939e+07,id1,0.549564,1.000000,NaN,NaN


## Explore Each of the IDs

In [13]:
file_name = "test_normal.csv"

id_dic, aid_signal_tups, max_start_time, min_end_time = reformat_synCAN_dataset(file_name, _, True)
print(aid_signal_tups, max_start_time, min_end_time)

[(1, 0), (1, 1), (2, 0), (2, 1), (2, 2), (3, 0), (3, 1), (4, 0), (5, 0), (5, 1), (6, 0), (6, 1), (7, 0), (7, 1), (8, 0), (9, 0), (10, 0), (10, 1), (10, 2), (10, 3)] 85508963.0 90009365.0


In [14]:
id_dic["id1"]

{'Signal1_of_ID': [array([85508925.3252, 85508940.3252, 85508955.3252, ..., 90009362.9808,
         90009377.9808, 90009392.9808]),
  array([0.62142346, 0.62142346, 0.62142346, ..., 0.54940344, 0.54948349,
         0.54956354])],
 'Signal2_of_ID': [array([85508925.3252, 85508940.3252, 85508955.3252, ..., 90009362.9808,
         90009377.9808, 90009392.9808]),
  array([0.75, 1.  , 0.  , ..., 0.5 , 0.75, 1.  ])]}

## Interpolate signals

In [15]:
signal_multivar_ts, timepts = dic_to_mv_signal_timeseries(id_dic, max_start_time, min_end_time, min_hz_msgs=100)

## Experiments on a Single Capture File (Distribution-Based)

In [17]:
signal_multivar_ts.shape

(450041, 20)

In [18]:
timepts

array([85508963., 85508973., 85508983., ..., 90009343., 90009353.,
       90009363.])

In [19]:
aid_signal_tups

[(1, 0),
 (1, 1),
 (2, 0),
 (2, 1),
 (2, 2),
 (3, 0),
 (3, 1),
 (4, 0),
 (5, 0),
 (5, 1),
 (6, 0),
 (6, 1),
 (7, 0),
 (7, 1),
 (8, 0),
 (9, 0),
 (10, 0),
 (10, 1),
 (10, 2),
 (10, 3)]

In [20]:
np.diff(timepts)

array([10., 10., 10., ..., 10., 10., 10.])

## Partition Time Series Attack

In [39]:
file_name = "test_normal.csv"

id_dic, aid_signal_tups, max_start_time, min_end_time = reformat_synCAN_dataset(file_name, _, True)
print(aid_signal_tups, max_start_time, min_end_time)

[(1, 0), (1, 1), (2, 0), (2, 1), (2, 2), (3, 0), (3, 1), (4, 0), (5, 0), (5, 1), (6, 0), (6, 1), (7, 0), (7, 1), (8, 0), (9, 0), (10, 0), (10, 1), (10, 2), (10, 3)] 85508963.0 90009365.0


## Interpolate Signals

In [42]:
sampling_frequency = 100
signal_multivar_ts, timepts = dic_to_mv_signal_timeseries(id_dic, max_start_time, min_end_time, min_hz_msgs=sampling_frequency)

In [43]:
signal_multivar_ts.shape

(450041, 20)

## Experiments Comparing Distributions

In [30]:
window = 10
offset = 1

partition_testing = process_multivariate_signals(signal_multivar_ts, aid_signal_tups, window, offset) # Partition time series
print("intervals: ", len(partition_testing))

# display(partition_testing[0])
# display(partition_testing[1])

corr_matrices_testing = compute_correlation_matrices(partition_testing) # Compute Correlations

total_length = (int(np.ceil(timepts[-1])) - int(np.ceil(timepts[0])))/1000
print("total length (s): ", total_length) 

intervals_testing = create_time_intervals(max_start_time, min_end_time, window*100, offset*100)
# print(len(intervals_testing), intervals_testing[-10:])

tp, fp, fn, tn = 0, 0, 0, 0

for index_interval in tqdm(range(len(intervals_testing))):
    
    # print(index_interval)

    # Get correlation matrices
    corr_sample_testing = upper(corr_matrices_testing[index_interval])
     
    # Do hypothesis test
    mannwhitneyu_test = mannwhitneyu(corr_sample_training, corr_sample_testing)
    # print((index_interval, len(corr_sample_training), len(corr_sample_testing), mannwhitneyu_test[0], mannwhitneyu_test[1]))

    # print(intervals_testing[index_interval][0], intervals_testing[index_interval][1], attack_metadata[attack_metadata_keys[0]]["injection_interval"][0], attack_metadata[attack_metadata_keys[0]]["injection_interval"][1])

    if mannwhitneyu_test[1] <= 0.05: # positive detection
        # if ((intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[0]]["injection_interval"][0] and intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[0]]["injection_interval"][0])
        #        or (intervals_testing[index_interval][0] > attack_metadata[attack_metadata_keys[0]]["injection_interval"][0] and intervals_testing[index_interval][1] < attack_metadata[attack_metadata_keys[0]]["injection_interval"][1])
        #            or (intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[0]]["injection_interval"][1] and intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[0]]["injection_interval"][1])):
        #     tp += 1
        # else:
        fp += 1
    else: # negative detection
        # if ((intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[0]]["injection_interval"][0] and intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[0]]["injection_interval"][0])
        #        or (intervals_testing[index_interval][0] > attack_metadata[attack_metadata_keys[0]]["injection_interval"][0] and intervals_testing[index_interval][1] < attack_metadata[attack_metadata_keys[0]]["injection_interval"][1])
        #            or (intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[0]]["injection_interval"][1] and intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[0]]["injection_interval"][1])):
        #     fn += 1
        # else:
        tn += 1
            
# precision
if tp + fp != 0:            
    precision = tp/(tp + fp)
else:
    precision = np.nan

# recall
if tp + fn != 0:
    recall = tp/(tp + fn)
else:
    recall = np.nan

# f1
if precision + recall != 0:
    f1 = 2*((precision*recall)/(precision + recall))

else:
    f1 = np.nan

# fpr
if fp + tn != 0:
    fpr = fp/(fp + tn)
else:
    fpr = np.nan

# fnr
if fn + tp != 0:
    fnr = fn/(fn + tp)
else:
    fnr = np.nan

# mcc
if (tp+fp == 0) or (tp+fn == 0) or (tn+fp == 0) or (tn+fn == 0):
    mcc = (tp*tn) - (fp*fn)
else:
    mcc = (tp*tn - fp*fn)/(math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))

print(f"tp: {tp}, tn: {tn}, fp: {fp}, fn: {fn}")
print(f"precision: {precision:.3f}, recall: {recall:.3f}, f1: {f1:.3f}, fpr: {fpr:.3f}, fnr: {fnr:.3f}, mcc: {mcc:.3f}")
print(f"positive_intervals: {tp+fn:.3f}, negative_intervals: {tn+fp:.3f}\n")

intervals:  44996
total length (s):  4500.4


100%|██████████| 44996/44996 [00:12<00:00, 3574.69it/s]

tp: 0, tn: 25106, fp: 19890, fn: 0
precision: 0.000, recall: nan, f1: nan, fpr: 0.442, fnr: nan, mcc: 0.000
positive_intervals: 0.000, negative_intervals: 44996.000



In [57]:
intervals_testing[-1]
# len(corr_matrices_testing)

(67506083.0, 67507025.0)

## Hypothesis Testing (All Attacks)

In [58]:
with open("/home/cloud/Projects/CAN/actt/data/SynCAN-Dataset/" + "dic_attack_intervals.json", "r") as fp:
    attack_metadata = json.load(fp)

attack_metadata_keys = list(attack_metadata.keys())
display(attack_metadata_keys)

['test_plateau.csv',
 'test_playback.csv',
 'test_suppress.csv',
 'test_flooding.csv',
 'test_continuous.csv']

In [59]:
display(attack_metadata['test_plateau.csv'])

for interval in attack_metadata['test_plateau.csv']:
    print(interval[1] - interval[0])

#attack_metadata['test_plateau.csv'][0][1] - attack_metadata['test_plateau.csv'][0][0]

[[67537573.9491, 67543212.5544],
 [67583022.5544, 67587191.7058],
 [67626329.2547, 67631726.6324],
 [67659992.2329, 67664968.7447],
 [67705122.5544, 67711720.6978],
 [67748279.3292, 67755660.0302],
 [67797168.6874, 67805239.2455],
 [67838863.9985, 67845463.3372],
 [67868300.0494, 67872810.3555],
 [67909407.8584, 67914702.5544],
 [67936260.229, 67944196.2129],
 [67983217.4205, 67987670.4139],
 [68018806.2129, 68026993.2948],
 [68064918.2239, 68073030.5126],
 [68109892.4205, 68114887.1873],
 [68138626.2129, 68143842.5544],
 [68174908.4883, 68180262.5613],
 [68203715.4139, 68211634.3061],
 [68246468.0408, 68253966.8615],
 [68293417.1322, 68298532.9226],
 [68337555.975, 68342595.2881],
 [68377306.2129, 68385612.5544],
 [68412650.4139, 68419489.8597],
 [68442728.2221, 68448157.4205],
 [68481817.7363, 68489692.4205],
 [68528347.4205, 68535500.8989],
 [68565253.7535, 68572062.4963],
 [68599498.8265, 68606396.07],
 [68633023.7535, 68639802.7876],
 [68675342.2033, 68682286.4468],
 [68707568.877

5638.6052999943495
4169.151399999857
5397.377700001001
4976.511800006032
6598.143399998546
7380.701000005007
8070.558100000024
6599.338699996471
4510.306099995971
5294.695999994874
7935.983899995685
4452.993400007486
8187.081900000572
8112.288699999452
4994.766800001264
5216.3414999991655
5354.072999998927
7918.8921999931335
7498.820699989796
5115.790399998426
5039.313100010157
8306.341499999166
6839.4457999914885
5429.198399990797
7874.684199988842
7153.47840000689
6808.74279999733
6897.243499994278
6779.03409999609
6944.243499994278
5189.434999987483
6553.150600001216
5218.966000005603
7857.902500003576
5728.206100001931
6704.57840000093
4287.4733999967575
6389.856299996376
6014.659199997783
6719.441699996591
7991.9201999902725
6912.715599998832
8294.92460000515
8294.0298999995
5666.956000000238
7213.631999999285
5339.006999999285
6509.659000009298
7288.28710000217
8078.469899997115
7527.224399998784
5069.127599999309
7268.0162999928
5699.340099990368
6812.177599996328
4273.050600007

In [53]:
window = 10
offset = 1
signals_training = corr_matrices_training[0].columns.values

# print(signals_training)

for index_attack in range(len(attack_metadata_keys)):

    print("Processing: ", attack_metadata_keys[index_attack])

    # signal_multivar_ts, timepts, aid_signal_tups = from_capture_to_time_series(testing_captures[index_attack], ground_truth_dbc_path) 

    id_dic, aid_signal_tups, max_start_time, min_end_time = reformat_synCAN_dataset(attack_metadata_keys[index_attack], _, True)
    signal_multivar_ts, timepts = dic_to_mv_signal_timeseries(id_dic, max_start_time, min_end_time, min_hz_msgs=10)
    
    partition_testing = process_multivariate_signals(signal_multivar_ts, aid_signal_tups, window, offset) # Partition time series
    print("intervals: ", len(partition_testing))
    
    # display(partition_testing[0])
    # display(partition_testing[1])

    corr_matrices_testing = compute_correlation_matrices(partition_testing) # Compute correlations
    
    total_length = (int(np.ceil(timepts[-1])) - int(np.ceil(timepts[0])))/1000 
    print("total length (s): ", total_length)
    intervals_testing = create_time_intervals(max_start_time, min_end_time, window*100, offset*100)
    # print(len(intervals_testing), intervals_testing[:5], intervals_testing[-5:], "\n")

    # print("intervals testing: ", intervals_testing)
    
    tp, fp, fn, tn = 0, 0, 0, 0

    for index_interval in range(len(intervals_testing)):

        # Get correlation matrices
        corr_sample_testing = upper(corr_matrices_testing[index_interval])

        # Do hypothesis test
        mannwhitneyu_test = mannwhitneyu(corr_sample_training, corr_sample_testing)
        # print((index_interval, len(corr_sample_training), len(corr_sample_testing), mannwhitneyu_test[0], mannwhitneyu_test[1]))

        if mannwhitneyu_test[1] <= 0.05: # positive detection
            # if ((intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][0] and intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][0])
            #        or (intervals_testing[index_interval][0] > attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][0] and intervals_testing[index_interval][1] < attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][1])
            #            or (intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][1] and intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][1])):
            if (binary_search(intervals_testing[index_interval][0], attack_metadata[attack_metadata_keys[index_attack]])) or (binary_search(intervals_testing[index_interval][1], attack_metadata[attack_metadata_keys[index_attack]])):
                tp += 1
            else:
                fp += 1
        else: # negative detection
            # if ((intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][0] and intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][0])
            #        or (intervals_testing[index_interval][0] > attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][0] and intervals_testing[index_interval][1] < attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][1])
            #            or (intervals_testing[index_interval][0] < attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][1] and intervals_testing[index_interval][1] > attack_metadata[attack_metadata_keys[index_attack]]["injection_interval"][1])):
            if (binary_search(intervals_testing[index_interval][0], attack_metadata[attack_metadata_keys[index_attack]])) or (binary_search(intervals_testing[index_interval][1], attack_metadata[attack_metadata_keys[index_attack]])):    
                fn += 1
            else:
                tn += 1
    # precision
    if tp + fp != 0:            
        precision = tp/(tp + fp)
    else:
        precision = np.nan
        
    # recall
    if tp + fn != 0:
        recall = tp/(tp + fn)
    else:
        recall = np.nan
        
    # f1
    if precision + recall != 0:
        f1 = 2*((precision*recall)/(precision + recall))
        
    else:
        f1 = np.nan
        
    # fpr
    if fp + tn != 0:
        fpr = fp/(fp + tn)
    else:
        fpr = np.nan

    # fnr
    if fn + tp != 0:
        fnr = fn/(fn + tp)
    else:
        fnr = np.nan

    # mcc
    if (tp+fp == 0) or (tp+fn == 0) or (tn+fp == 0) or (tn+fn == 0):
        mcc = (tp*tn) - (fp*fn)
    else:
        mcc = (tp*tn - fp*fn)/(math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))

    print(f"tp: {tp}, tn: {tn}, fp: {fp}, fn: {fn}")
    print(f"precision: {precision:.3f}, recall: {recall:.3f}, f1: {f1:.3f}, fpr: {fpr:.3f}, fnr: {fnr:.3f}, mcc: {mcc:.3f}")
    print(f"positive_intervals: {tp+fn:.3f}, negative_intervals: {tn+fp:.3f}\n")


Processing:  test_plateau.csv
intervals:  44996
total length (s):  4500.4
tp: 2937, tn: 22119, fp: 15362, fn: 4578
precision: 0.161, recall: 0.391, f1: 0.228, fpr: 0.410, fnr: 0.609, mcc: -0.014
positive_intervals: 7515.000, negative_intervals: 37481.000

Processing:  test_playback.csv
intervals:  44996
total length (s):  4500.4
tp: 1942, tn: 22413, fp: 17962, fn: 2679
precision: 0.098, recall: 0.420, f1: 0.158, fpr: 0.445, fnr: 0.580, mcc: -0.015
positive_intervals: 4621.000, negative_intervals: 40375.000

Processing:  test_suppress.csv
intervals:  44996
total length (s):  4500.4
tp: 3675, tn: 20427, fp: 15931, fn: 4963
precision: 0.187, recall: 0.425, f1: 0.260, fpr: 0.438, fnr: 0.575, mcc: -0.010
positive_intervals: 8638.000, negative_intervals: 36358.000

Processing:  test_flooding.csv
intervals:  44996
total length (s):  4500.4
tp: 2547, tn: 22530, fp: 16134, fn: 3785
precision: 0.136, recall: 0.402, f1: 0.204, fpr: 0.417, fnr: 0.598, mcc: -0.011
positive_intervals: 6332.000, nega